In [16]:
import ccxt
import os
from dotenv import load_dotenv
import time
import schedule
import pandas as pd
import numpy as np
from datetime import date, datetime, timezone, tzinfo

In [2]:
load_dotenv()

# Retrieve API keys from environment variables
xP_KEY = os.getenv('xP_KEY')
xP_SECRAT = os.getenv('xP_SECRET')

kC_KEY = os.getenv('kC_KEY')
kC_SECRET = os.getenv('kC_SECRAT')
kC_PASSWORD = os.getenv('kC_PASSWORD')


In [3]:
# Initialize the KuCoin exchange object
kucoin = ccxt.kucoin({
    'enableRateLimit': True,
    'apiKey': kC_KEY,
    'secret': kC_SECRET,
    'password': kC_PASSWORD
})
phemex = ccxt.phemex({
    'enableRateLimit': True,
    'apiKey': xP_KEY,
    'secret': xP_SECRAT
})
symbol = 'BTCUSD'
symbol_str = "STRKUSDTM"

# Helping Function

In [4]:

import json

from kucoin_futures.client import Trade

def get_pos_size(sy):        
    client = Trade(key=kC_KEY, secret=kC_SECRET, passphrase=kC_PASSWORD)
    positions = client.get_position_details(symbol=sy)
    if positions['currentQty'] and positions['isOpen']:
        return positions['currentQty'],positions['isOpen']
    return 0,None

def ask_bid():
    order_book = phemex.fetch_order_book(symbol=symbol)
    return order_book['asks'][0][0], order_book['bids'][0][0]
get_pos_size(sy=symbol_str)


(2, True)

# Daily SMA 20 Days

In [5]:
 #! FIND DAILY SMA PER 20 DAYS

def sma_20d():
    print("starting 20 Days ...")

    timeframe = '1d'
    num_bars = 100
    bars = phemex.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=num_bars)
    # print(" Got the Bars: ",bars)
    data_20days = pd.DataFrame(bars, columns=['timestep', 'open', 'high', 'low', 'close', 'volume'])
    data_20days['date'] = pd.to_datetime(data_20days['timestep'], unit='ms')
    data_20days.drop(columns=['timestep'], inplace=True)
    data_20days['SMA_20'] = data_20days['close'].rolling(window=20).mean()

    bid = ask_bid()[1] 
    data_20days.loc[data_20days['SMA_20'] > bid, 'SIGNAL'] = 'SELL'
    data_20days.loc[data_20days['SMA_20'] < bid, 'SIGNAL'] = 'BUY'

    if data_20days['SIGNAL'].iloc[-1] == 'SELL':
        print("Opening as sell ...")
    else:
        print('Opening as Buy ...')
    
    return data_20days

sma_20d()


starting 20 Days ...
Opening as sell ...


,open,high,low,close,volume,date,SMA_20,SIGNAL
0,63139.7,63290.0,58206.9,60223.7,232487908.0,2024-06-24,NaN,NaN
1,60220.8,62372.2,60193.2,61761.2,80158692.0,2024-06-25,NaN,NaN
2,61761.2,62449.0,60660.9,60790.0,58474523.0,2024-06-26,NaN,NaN
3,60800.2,62373.9,60541.2,61628.9,63089257.0,2024-06-27,NaN,NaN
4,61629.0,62159.1,59944.0,60331.5,79926174.0,2024-06-28,NaN,NaN
...,...,...,...,...,...,...,...,...
95,65175.6,66490.8,64828.6,65742.1,97440115.0,2024-09-27,61027.830,BUY
96,65751.0,66261.0,65387.2,65843.5,24417441.0,2024-09-28,61574.185,SELL
97,65836.5,66088.4,65412.5,65600.2,26061657.0,2024-09-29,62000.810,SELL
98,65601.3,65614.1,62792.0,63312.8,128679319.0,2024-09-30,62282.670,SELL


#  15 Minutes SMA

In [6]:
 #! Define the SMA for the 15 Minutes With Prices bp_1, bp_2, sp_1, sp_2

def sma_15min():
    print("starting sma 15 Minuts  ...")

    timeframe = '15m'
    num_bars = 100
    bars = phemex.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=num_bars)
    # # print(" Got the Bars: ",bars)
    data_15m = pd.DataFrame(bars, columns=['timestep', 'open', 'high', 'low', 'close', 'volume'])
    data_15m['date'] = pd.to_datetime(data_15m['timestep'], unit='ms')
    data_15m.drop(columns=['timestep'], inplace=True)
    data_15m['SMA_15m'] = data_15m['close'].rolling(window=20).mean()

    bid = ask_bid()[1] 
        
    data_15m['bp_1'] = data_15m['SMA_15m'] * 1.001
    data_15m['bp_2'] = data_15m['SMA_15m'] * 0.997
    data_15m['sp_1'] = data_15m['SMA_15m'] * 0.999
    data_15m['sp_2'] = data_15m['SMA_15m'] * 1.003

    return data_15m

sma_15min()

starting sma 15 Minuts  ...


,open,high,low,close,volume,date,SMA_15m,bp_1,bp_2,sp_1,sp_2
0,63568.7,63649.5,63496.5,63505.6,603272.0,2024-10-01 03:15:00,NaN,NaN,NaN,NaN,NaN
1,63518.8,63747.3,63509.5,63726.0,869506.0,2024-10-01 03:30:00,NaN,NaN,NaN,NaN,NaN
2,63714.9,63731.0,63653.2,63711.8,414188.0,2024-10-01 03:45:00,NaN,NaN,NaN,NaN,NaN
3,63712.1,63846.1,63637.6,63654.7,2498898.0,2024-10-01 04:00:00,NaN,NaN,NaN,NaN,NaN
4,63654.7,63762.7,63654.7,63744.5,418450.0,2024-10-01 04:15:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95,61771.0,61822.7,61674.8,61744.5,513644.0,2024-10-02 03:00:00,61154.265,61215.419265,60970.802205,61093.110735,61337.727795
96,61744.2,61811.4,61664.2,61677.1,353492.0,2024-10-02 03:15:00,61197.565,61258.762565,61013.972305,61136.367435,61381.157695
97,61677.1,61860.7,61632.5,61834.4,2827656.0,2024-10-02 03:30:00,61257.730,61318.987730,61073.956810,61196.472270,61441.503190
98,61838.2,61841.5,61746.5,61749.7,528807.0,2024-10-02 03:45:00,61308.890,61370.198890,61124.963330,61247.581110,61492.816670


# Bot

In [7]:
def bot():
    data_15m = sma_15min()
    data_20d = sma_20d()
    ask,bid = ask_bid()

    # * Define OPEN Order
    signal = data_20d.iloc[-1]['SIGNAL']
    
    open_size = 1
    
    in_pos = pnl_close()[1]


    if not in_pos:
        if signal == "BUY":
            print("Make order as BUY !!")
            bp_1 = data_15m.iloc[-1]['bp_1']
            bp_2 = data_15m.iloc[-1]['bp_2']
            print(f"bp_1: {bp_1} bp_2: {bp_2}")
            # TODO: Make Buy Order Here ...
        else:
            print("Make order as SELL !!")
            sp_1 = data_15m.iloc[-1]['sp_1']
            sp_2 = data_15m.iloc[-1]['sp_2']
            print(f"sp_1: {sp_1} sp_2: {sp_2}")
            # TODO: Make Sell Order Here ... 
    else:
        print("Already In Position, not making new Position")
bot()

starting sma 15 Minuts  ...
starting 20 Days ...
Opening as sell ...


NameError: name 'pnl_close' is not defined

In [14]:
target = 25

def kill_switch():
    print("Starting the Kill switching ...")
    # TODO: Implement the Kill Switch
    pnl_c, _ ,side = pnl_close_kucoin() #! pnl_close, open_pos, Side-long,short
    size_for_kill,open_p, = get_pos_size(sy=symbol_str)

    print(f"open position: {open_p} side: {side} Size postion: {size_for_kill}")
    
    ask,bid = ask_bid()
    while open_p:
        print(f"Killing the size {size_for_kill}")

        if side == 'long':
            print(f" -- -- Make the order for Sell at {ask} With quntety of {size_for_kill} of {symbol_str}")
            print("=== Sleeping 30 Sec =====")
            time.sleep(30)
            # TODO: Existing the SELL the Order. At Ask.

        elif side == 'short':
            print(f" -- -- Make the order for BUY at {bid} With quntety of {size_for_kill} of {symbol_str}")
            print("=== Sleeping 30 Sec =====")
            # TODO: Existing the BUY Order. At bid.
        
        else:
            print(f" -- Position is not Sure, Not kill switching ...")


def pnl_close_kucoin():
    print("Starting PNL ...")
    
    # Example of different parameters

    poc_dic = kucoin.fetch_ticker(symbol='STRK/USDT')
    perc = poc_dic['percentage']
    diff = float(poc_dic['info']['changePrice'])
    current_price = poc_dic['last']
    entry_price = float(poc_dic['open']) 
    side = 'long' if (diff >= 0 and current_price >= entry_price) else 'short'
    if diff != 0:
        side = 'long' if diff > 0 and current_price > entry_price else 'short'
    else:
        side = 'none'

    print(f"Difference: {diff}, Current Price: {current_price}, Entry Price: {entry_price}, Side: {side}, Percentage: {perc}%")


    pnlclose = False
    in_pos = False
    if perc > 0:
        # * We are in a winning position (profit)
        if perc > target:
            # ! Closing the position as target is hit
            print("In Profit also Hit the Target ...")
            pnlclose = True
            kill_switch()
        else:
            print("In Profit but  not hit the target, so wait !!")
    else:
        in_pos = True
        print("We are not in Profit yet.")
    print("Finishing the PNL close ...")

    return pnlclose, in_pos,side

# print(pnl_close_phemex())
# pnl_close_kucoin()
kill_switch()

Starting the Kill switching ...
Starting PNL ...
Difference: -0.0676, Current Price: 0.39199, Entry Price: 0.45959, Side: short, Percentage: -14.7%
We are not in Profit yet.
Finishing the PNL close ...
open position: True side: short Size postion: 2


RequestTimeout: phemex GET https://api.phemex.com/v1/md/fullbook?symbol=BTCUSD

In [26]:
def hello_yash():
    print("hello yash")

schedule.every(5).seconds.do(hello_yash)

while True:
    # schedule.run_pending()
    hello_yash()
    time.sleep(5)  # Add a sleep to prevent a busy-wait loop

hello yash


KeyboardInterrupt: 

[[15, 0, 0, 0, 0],
 [7, 14, 0, 0, 0],
 [6, 8, 13, 0, 0],
 [0, 5, 9, 12, 0],
 [0, 0, 4, 10, 11]]